## Creating mesh (with gmsh)

In [ ]:
import sys
from pathlib import Path
import numpy as np

creating a domain with pre-cracks

In [ ]:
try :
    import gmsh
except :
    print('gmsh python module necessary')
    print('consider installing gmsh sdk from http://gmsh.info')
    sys.exit()  

gmsh.initialize(sys.argv)
gmsh.option.setNumber("General.Terminal", 1)

gmsh.model.add("square with cracks")

lc = 0.05

gmsh.option.setNumber("Mesh.Algorithm", 6);
gmsh.option.setNumber("Mesh.CharacteristicLengthMin", lc);
gmsh.option.setNumber("Mesh.CharacteristicLengthMax", lc);
gmsh.option.setNumber("Mesh.MshFileVersion",4) 


surf1 = gmsh.model.occ.addRectangle(0, 0, 0, 1, 1)

pt1 = gmsh.model.occ.addPoint(0.2, 0.2, 0)
pt2 = gmsh.model.occ.addPoint(0.4, 0.4, 0)
line1 = gmsh.model.occ.addLine(pt1, pt2)
pt3 = gmsh.model.occ.addPoint(0.6, 0.1, 0)
pt4 = gmsh.model.occ.addPoint(0.1, 0.3, 0)
line2 = gmsh.model.occ.addLine(pt3, pt4)

o, m = gmsh.model.occ.fragment([(2,surf1)], [(1,line1), (1,line2)])
gmsh.model.occ.synchronize()

# m contains, for each input entity (surf1, line1 and line2), the child entities
# (if any) after the fragmentation, as lists of tuples. To apply the crack
# plugin we group all the intersecting lines in a physical group

ns_=m[0][0][1]
nsl_=[item[1] for item in gmsh.model.getBoundary([(2,ns_)])]

nl_=[]
for v_ in o:
    if v_[0] == 1 and v_[1] not in nsl_:
        nl_.append(v_[1])

lup_=[item[1] for item in gmsh.model.getEntitiesInBoundingBox(-0.1,0.999,-1,1.1,1.1,1.,dim=1)]
ldown_=[item[1] for item in gmsh.model.getEntitiesInBoundingBox(-0.1,-0.1,-1,1.1,0.001,1.,dim=1)]

surf_ = gmsh.model.addPhysicalGroup(2,[ns_])
fis_ = gmsh.model.addPhysicalGroup(1, nl_)
up_  = gmsh.model.addPhysicalGroup(1, lup_)
down_ = gmsh.model.addPhysicalGroup(1, ldown_)

#gmsh.fltk.run()

gmsh.model.mesh.generate(2)

gmsh.write("crack.msh")

gmsh.finalize()

In [ ]:
!gmsh crack.msh

## Creating sample

In [ ]:
from pylmgc90 import pre

dim = 2

bodies = pre.avatars()
mat    = pre.materials()
mod    = pre.models()
svs    = pre.see_tables()
tacts  = pre.tact_behavs()
post   = pre.postpro_commands()

In [ ]:
steel = pre.material(name='Steel', materialType='ELAS', elas='standard',
                     young=1.3e8, nu=0.3, anisotropy='isotropic', density=2500.)  
mat.addMaterial(steel)

m2Dl = pre.model(name='M2DNL', physics='MECAx', element='T3xxx', dimension=dim, external_model='MatL_',
                 kinematic='large', material='neoh_', anisotropy='iso__', mass_storage='lump_',formulation='TotaL')
mod.addModel(m2Dl)

In [ ]:
# read the mesh file
mesh_= pre.readMesh('crack.msh',dim)

# create a sample
sample = pre.buildMeshedAvatar(mesh=mesh_, model=m2Dl, material=steel)

In [ ]:
# manage cracks in the mesh (add nodes and elements along the crack)
new_sample,new_groups = pre.crackMeshedAvatar2D(body=sample, crackgroup=str(fis_))

In [ ]:
# explode the continuous domain 
new_bodies=pre.explodeMeshedAvatar2D(body=new_sample, nbPoints=2, color='REDxx', w=[0.25,0.75]) 

In [ ]:
# apply boundary conditions and contactors
d_=[]
u_=[]

for body in new_bodies:    
    if body.hasGroup(str(down_)):
        body.imposeDrivenDof(group=str(down_)  ,component=2     , dofty='vlocy', ct=5.)
        d_.append((body,str(down_)))        

    if body.hasGroup(str(up_)):
        body.imposeDrivenDof(group=str(up_)  ,component=2     , dofty='vlocy', ct=-5.)
        u_.append((body,str(up_)))

    if body.hasGroup(str(fis_)):
        body.addContactors(group=str(fis_), shape='CLxxx', color='frac_', weights=[0.25,0.75]) #, reverse=True)

    for grp_ in new_groups:
        if body.hasGroup(grp_) :  
            body.addContactors(group=str(grp_), shape='ALpxx', color='frac_') #,reverse=True)

    bodies += body

all_=[u_,d_]

# new fashion

l2 = pre.tact_behav(name='mal__', law='MAL_CZM', dyfr=0.6, stfr=0.8,
                    cn=1.e14, s1=5.1e6, G1=0.15,
                    ct=1.e14, s2=8.3e6, G2=0.35) 
tacts += l2

l1 = pre.tact_behav( name='xxxxx', law='GAP_SGR_CLB',fric=0.6)
tacts += l1

s2 = pre.see_table(CorpsCandidat   ='MAILx', candidat   ='CLxxx', colorCandidat   ='REDxx',
                   CorpsAntagoniste='MAILx', antagoniste='ALpxx', colorAntagoniste='REDxx',
                   behav=l2, halo=1.2*lc,alert=lc/10)
svs += s2

s1 = pre.see_table(CorpsCandidat   ='MAILx', candidat   ='CLxxx', colorCandidat   ='frac_',
                   CorpsAntagoniste='MAILx', antagoniste='ALpxx', colorAntagoniste='frac_',
                   behav=l1, halo=1.2*lc,alert=lc/10)
svs += s1


# ecriture des fichiers
datbox = Path('./DATBOX')
datbox.mkdir(exist_ok=True)


post = pre.postpro_commands()
post.addCommand(pre.postpro_command(name = 'NEW MECAx SETS', mecax_sets=all_))
post.addCommand(pre.postpro_command(name = 'Dep EVOLUTION', step = 1 ))
post.addCommand(pre.postpro_command(name = 'Fint EVOLUTION', step = 1 ))
post.addCommand(pre.postpro_command(name = 'SOLVER INFORMATIONS', step = 1 ))

pre.writeDatbox(dim, mat, mod, bodies, tacts, svs, post=post, datbox_path=datbox, gravy=[0.,0.,0.])

In [ ]:
try:
    pre.visuAvatars(bodies)
except:
    pass

## Computation

In [ ]:
import numpy as np
# importing chipy module
from pylmgc90 import chipy

# Initializing
chipy.Initialize()

# checking/creating mandatory subfolders
chipy.checkDirectories()

restart = 0

# logMes
chipy.utilities_DisableLogMes()

#
# defining some variables
#

# space dimension
dim = 2

# modeling hypothesis ( 1 = plain strain, 2 = plain stress, 3 = axi-symmetry)
mhyp = 1

# time evolution parameters
dt = 1.e-5
nb_steps = 200

# theta integrator parameter
theta = 0.5

# deformable  yes=1, no=0
deformable = 1

# interaction parameters
Rloc_tol = 5.e-2

# nlgs parameters
tol = 1e-5
relax = 1.0
norm = 'Quad '
gs_it1 = 500
gs_it2 = 20
solver_type='Stored_Delassus_Loops         '

# write parameter
freq_write   = 10

# display parameters
freq_display = 10

nb_CL_by_Edge = 2
chipy.CLxxx_SetNbNodesByCLxxx(nb_CL_by_Edge)


In [ ]:
#
# read and load
#

# Set space dimension
chipy.SetDimension(dim,mhyp)
#
chipy.utilities_logMes('INIT TIME STEPPING')
chipy.TimeEvolution_SetTimeStep(dt)
chipy.Integrator_InitTheta(theta)
#
chipy.utilities_logMes('READ DATBOX')
chipy.ReadDatbox(deformable=True)


# to display boundary conditions...
dofstatus=[]
# here the nodal field is generated by a built-in function
nb=chipy.mecaMAILx_GetNbMecaMAILx()
for i_ in range(1,nb+1):
     dofstatus.append(chipy.mecaMAILx_GetDofStatus(i_))

#
# open display & postpro
#

chipy.utilities_logMes('DISPLAY & WRITE')
chipy.OpenDisplayFiles(restart+1)
chipy.OpenPostproFiles()

In [ ]:
#
# simulation part ...
#

# ... calls a simulation time loop
# since constant compute elementary mass matrices once
chipy.utilities_logMes('COMPUTE MASS')
chipy.ComputeMass()

# since constant compute elementary stiffness matrices once
chipy.utilities_logMes('COMPUTE STIFFNESS')
chipy.ComputeBulk()

# since constant compute iteration matrix once
chipy.utilities_logMes('ASSEMB KT')
chipy.AssembleMechanicalLHS()

ofile = open('./cracklength.txt','w')

for k in range(1, nb_steps + 1, 1):
    #
    chipy.utilities_logMes('INCREMENT STEP')
    chipy.IncrementStep()
    #
    chipy.utilities_logMes('COMPUTE Fext')
    chipy.ComputeFext()
    #
    chipy.utilities_logMes('COMPUTE Fint')
    chipy.ComputeBulk()
    #
    chipy.utilities_logMes('ASSEMB RHS')
    chipy.AssembleMechanicalRHS()
    #
    chipy.utilities_logMes('COMPUTE Free Vlocy')
    chipy.ComputeFreeVelocity()
    #
    chipy.utilities_logMes('SELECT PROX TACTORS')
    chipy.SelectProxTactors()
    #
    chipy.utilities_logMes('RESOLUTION' )
    chipy.RecupRloc()
    #
    chipy.ExSolver(solver_type, norm, tol, relax, gs_it1, gs_it2)
    chipy.UpdateTactBehav()
    #
    chipy.StockRloc()
    #
    chipy.utilities_logMes('COMPUTE DOF, FIELDS, etc.')
    chipy.ComputeDof()
    #
    chipy.utilities_logMes('UPDATE DOF, FIELDS')
    chipy.UpdateStep()
    #
    chipy.utilities_logMes('WRITE OUT DOF')
    chipy.WriteOutDof(freq_write)
    #
    chipy.utilities_logMes('WRITE OUT GPV')
    chipy.WriteOutGPV(freq_write)
    #
    chipy.utilities_logMes('WRITE OUT Rloc')
    chipy.WriteOutVlocRloc(freq_write)
    #
    chipy.utilities_logMes('VISU & POSTPRO')

    if chipy.inter_handler_2D_getNb(chipy.CLALp_ID ) == 0:
        chipy.utilities_logMes('no CLALp contacts, impossible ! ')
        sys.exit()
        
    # crack and total length
    cl = 0.
    tl = 0.
  
    inters = chipy.getInteractions()
    
    betas = chipy.getInternalArray('beta', inters)

    ess   = chipy.getInternalArray('#taille_ele', inters)

    tl = np.sum( ess )
    cl = np.sum( (1.-betas)*ess )

    ofile.write('%12.5e %12.5e %12.5e \n' % (chipy.TimeEvolution_GetTime(),cl,tl))
      
    if not k%freq_display :    
        chipy.WriteDisplayFiles(freq=1,                    \
                                beta=('ptc', betas),     \
                                DrvDof=('mecafe','node',dofstatus))

    chipy.WritePostproFiles()


In [ ]:
ofile.close()

#
# close display & postpro
#
chipy.CloseDisplayFiles()
chipy.ClosePostproFiles()

# this is the end
chipy.Finalize()

In [ ]:
!paraview